In [ ]:
res = requests.get('https://www.twse.com.tw/fund/T86?response=csv&date=20191009&selectType=ALL')
for p in res.text.split('\n'):
    print(p)

In [ ]:
print(re.sub('[="]','',res.text.split('\n')[2]))

In [4]:
import requests
import os
from os import mkdir
from os.path import isdir
from datetime import datetime
import csv
class Crawler():
    def __init__(self,dir='data/chips'):
        # save in directory 'data'
        if not isdir(dir):
            mkdir(dir)
        self.dir = dir
    
    def writeCSV(self,file,rows):
        with open("{}/{}.csv".format(self.dir,file),'a') as f:
            writer = csv.writer(f,lineterminator='\n')
            writer.writerows(rows)

    def getChipByDate(self,date,file):
        twse = "https://www.twse.com.tw/fund/T86"
        para = {
            'response' : 'csv',
            'date' : date,
            'selectType' : 'ALL'
        }
        result = requests.get(twse,params=para)
        if not result.ok:
            raise ConnectionError
            return
        if '沒有符合條件的資料' in result.text:
            return
        rows = result.text.split('\n')[:-10]
        r_rows = [['#==#'+date]]
        for i in range(2,len(rows)):
            #row cleanup and split
            row = re.sub('[,"=]','',rows[i].replace(',"','_')).replace('_',',').split(',')
            r_rows.append(row)
        self.writeCSV(file,r_rows)

In [ ]:
c = Crawler()
s = c.getChipByDate('20191009')

In [ ]:
from datetime import datetime
from datetime import timedelta
import time
import csv
oneday = timedelta(days=1)
date = datetime(2019,10,9)
final = datetime(2012,5,2)
file = str(date)[:10].replace('-','')

while(date>final):
    c = Crawler()
    ds = str(date)[:10].replace('-','')
    s = c.getChipByDate(ds,file)
    print('{} - complete'.format(str(date)[:10]))
    time.sleep(1.2)
    date -= oneday

In [8]:
from datetime import datetime
from datetime import timedelta
import time
import csv
import sys
import re
startIndex =0
endIndex = 1000000
total_index = 0


s_dates = ''
with open('data/chips/20191009.csv','rb') as fin:
    while(1):
        fin.seek(startIndex,0)
        #print(fin.tell())
        data = fin.read(endIndex)
        rows=''
        try:
            rows = data.decode('utf-8').split('#==#')[1]
        except UnicodeDecodeError as err:
            print('fail')
            endIndex-=1
            continue
        except IndexError as ierr:
            break
        startIndex += len(rows.encode('utf-8'))+4
        if len(rows) <= 10:
            continue
        ss = rows.split('\n')
        if len(ss)>2:
            s_dates += ss[0]
print(s_dates)


fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


In [7]:
oneday = timedelta(days=1)
date = datetime(2019,10,9)
final = datetime(2012,5,2)
file = str(date)[:10].replace('-','')
while(date>final):
    if str(date)[:10].replace('-','') in s_dates:
        print(str(date)+'  exist')
        time.sleep(1.2)
        date -= oneday 
        continue
    c = Crawler()
    ds = str(date)[:10].replace('-','')
    s = c.getChipByDate(ds,file)
    print('{} - complete'.format(str(date)[:10]))
    time.sleep(1.2)
    date -= oneday    

2019-10-09 00:00:00  exist
2019-10-08 00:00:00  exist
2019-10-07 00:00:00  exist
2019-10-06 - complete
2019-10-05 - complete
2019-10-04 00:00:00  exist
2019-10-03 00:00:00  exist
2019-10-02 00:00:00  exist
2019-10-01 00:00:00  exist
2019-09-30 - complete
2019-09-29 - complete
2019-09-28 - complete
2019-09-27 00:00:00  exist
2019-09-26 00:00:00  exist
2019-09-25 00:00:00  exist
2019-09-24 00:00:00  exist
2019-09-23 00:00:00  exist
2019-09-22 - complete
2019-09-21 - complete
2019-09-20 00:00:00  exist
2019-09-19 00:00:00  exist
2019-09-18 00:00:00  exist
2019-09-17 00:00:00  exist
2019-09-16 00:00:00  exist
2019-09-15 - complete
2019-09-14 - complete
2019-09-13 - complete
2019-09-12 00:00:00  exist
2019-09-11 00:00:00  exist
2019-09-10 00:00:00  exist
2019-09-09 00:00:00  exist
2019-09-08 - complete
2019-09-07 - complete
2019-09-06 00:00:00  exist
2019-09-05 00:00:00  exist
2019-09-04 00:00:00  exist
2019-09-03 00:00:00  exist
2019-09-02 00:00:00  exist
2019-09-01 - complete
2019-08-31 -

ConnectionError: HTTPSConnectionPool(host='www.twse.com.tw', port=443): Max retries exceeded with url: /fund/T86?response=csv&date=20190127&selectType=ALL (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fb272aa80b8>: Failed to establish a new connection: [Errno 111] Connection refused'))